### DATA 620 - Project 1
Author: Kory Martin  
Date: 2/18/2024

### 1. Introduction

#### Project Instructions

<ul>
<li>For your first project, you are asked to identify and load a network dataset that has some categorical information available for each node.
<li>For each of the nodes in the dataset, calculate degree centrality and eigenvector centrality.
<li>Compare your centrality measures across your categorical groups.
</ul>
    

#### Project Plan
I chose to use the a movieLens dataset [here](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html), where we see users and the ratings that they made for a number of movies. Using this data, I am going to build a network of Users that have rated at least 15 of the same movies. This type of analysis can be used to identify similar users - if we consider that the movies that a user watches would suggest a level of similarity.


For this analysis, we will focus on answering the following questions for a network:

<ul>
    <li> How many nodes are in the graph?
    <li> What is the diameter of the Graph?    
    <li> What is the size of the Graph?
    <li> What is the average number of neighbors for nodes? 
    <li> What is the largest number of neighbors that any particular node has?
    <li> What node(s) has the largest number of neighbors?
    <li> What is the fewest number of neighbors that any particular node has?
    <li> What node(s) has the fewest number of neighbors?
</ul>

### 2. Import Libraries

In [11]:
import networkx as net
from networkx import algorithms as algo
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from statistics import mean
import os.path

### 3. Data Preparation

We begin by importing the data for the specific movie titles that were being reviewed. Although this ultimately isn't something that will be included in the final network analysis, it was something that I had initially thought I wanted to specifically analyze. Additionally, the ratings data will be imported

In [12]:
movies_data = pd.read_csv('data/movies.csv')
ratings_data = pd.read_csv('data/ratings.csv')

In [12]:
movies_data.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

Now that we have the movies and the ratings data imported, we will merge the two datasets so that we are able to track exactly which movie titles a specific user rated.

In [13]:
m1 = pd.merge(ratings_data, movies_data, how='left',
        left_on='movieId', right_on='movieId')

Finally, we will then create a more simplified version of our merged dataframe that will only include the data for the user, the movie title and the rating that they gave to that movie

In [14]:
m1_simple = m1.loc[:,['title','userId','rating']]

In [15]:
movie_titles = list(movies_data['title'].unique())

In [16]:
movie_raters = list(ratings_data['userId'].unique())

##### Create genres list

For the output, I want to expand the movies_data dataframe so that we have an entry for each title along with their genres.

In [17]:
movies_data_expanded = []

for i in range(len(movies_data)):

    movie_id = movies_data.loc[i,'movieId']
    movie_title = movies_data.loc[i,'title']

    genres = movies_data.loc[i,'genres'].split("|")
    for genre in genres:
        mini_dict = {}
        mini_dict['movieId'] = movie_id
        mini_dict['title'] = movie_title
        mini_dict['genre'] = genre
        movies_data_expanded.append(mini_dict)

In [18]:
movies_expanded_df = pd.DataFrame(movies_data_expanded)

In [19]:
movie_count = m1_simple.groupby('userId').count().reset_index()

In [20]:
movie_count['pct_watched'] = movie_count['title']/len(movie_titles)

In [73]:
movie_raters_subset = list(movie_count[movie_count['pct_watched'] <= .025]['userId'].unique())

In [70]:
503/len(movie_raters)

0.8245901639344262

In [89]:
m1_sub_titles = list(m1_simple[m1_simple['userId'].isin(movie_raters_subset)]['title'].unique())

##### Create movie pairs

This next block of code is where we will build out the movie pairs for our dataset. The code creates a filtered dataset for the specific user that we are looking at and then creates node relationships based on the movies that are watched by this user

In [91]:
if not os.path.exists('data/movie_pairs.csv'):

    movie_pairs = []
    for user in movie_raters_subset:

        m1_sub = m1_simple.loc[m1_simple['userId'] == user]
        movie_titles = list(m1_sub['title'].unique())
        for i in range(len(movie_titles)):

            movie1 = m1_sub.iloc[i]['title']
            j = i+1
            while j < len(movie_titles):
                movie2 = m1_sub.iloc[j]['title']
                j+=1
                movie_pair = (movie1, movie2)
                movie_pairs.append(movie_pair)

    movie_pairs_df = pd.DataFrame(movie_pairs) 
    movie_pairs_df.to_csv('data/movie_pairs.csv',index=False)
    
else:
    movie_pairs_df = pd.read_csv('data/movie_pairs.csv')

In [104]:
movie_pairs_df = movie_pairs_df.rename(columns={0:'movie1',1:'movie2'})

In [106]:
network_edges = []
for i in range(len(movie_pairs_df)):
    mini_tupt = (movie_pairs_df.loc[i,'movie1'],movie_pairs_df.loc[i,'movie2'])
    #print(mini_tupt)
    network_edges.append(mini_tupt)

### 3. Create Network

Now that we have the relationship data established, we will display the network of individuals and from there we can examine different measurements related to this network. 

Some of the questions we will look to examine are:

<ul>
    <li> How many nodes are in the graph?
    <li> What is the diameter of the Graph?    
    <li> What is the size of the Graph?
    <li> What is the average number of neighbors for nodes? 
    <li> What is the largest number of neighbors that any particular node has?
    <li> What node(s) has the largest number of neighbors?
    <li> What is the fewest number of neighbors that any particular node has?
    <li> What node(s) has the fewest number of neighbors?
</ul>

In [107]:
G = net.Graph()
G.add_nodes_from(m1_sub_titles)
G.add_edges_from(network_edges)
#net.draw(G,with_labels=True, font_size = 6, node_size=300)
#net.draw(G, font_size = 6, node_size=300)

### 4. Calculate the Centrality

#### Degree Centrality

In [108]:
degree_centrality = net.degree_centrality(G)

In [109]:
centrality_list = []
centrality_list.append(degree_centrality)

In [110]:
degree_centrality_df = pd.DataFrame(centrality_list).transpose().reset_index()

degree_centrality_df = degree_centrality_df.rename(columns={'index':'title',0:'centrality'})

#### Eigenvector Centrality

In [111]:
eigenvector_centrality = net.eigenvector_centrality(G)

In [112]:
centrality_list = []
centrality_list.append(eigenvector_centrality)

In [113]:
eigenvector_centrality_df = pd.DataFrame(centrality_list).transpose().reset_index()

eigenvector_centrality_df = eigenvector_centrality_df.rename(columns={'index':'title',0:'centrality'})

In [114]:
eigenvector_centrality_df.head()

title  centrality
0             Toy Story (1995)    0.047037
1      Grumpier Old Men (1995)    0.023855
2                  Heat (1995)    0.035039
3  Seven (a.k.a. Se7en) (1995)    0.042372
4   Usual Suspects, The (1995)    0.044370

In [ ]:
betweenness_centrality = net.betweenness_centrality(G)

### 5. Network Analysis

<b> a. How many nodes are included in the network?

In [115]:
len(list(G.nodes()))

4894

There are 600 nodes in this network - meaning that of our 610 users with ratings. This represents, 98% of the users in the network

<b> b. What is the diameter of the Graph?

In [ ]:
#net.diameter(G)

The diameter of the graph is 3

<b> c. What is the size of the Graph?

In [116]:
G.size()

1145704

There are a total of 1,145,704 connections amongst the users in this network

<b>d.  What is the average and median number of neighbors for nodes?

In [117]:
num_neighbors = [len(list(net.neighbors(G,user))) for user in list(G.nodes())]
numbers_group = [{'title':user, 'num_neighbors':len(list(net.neighbors(G,user)))} for user in list(G.nodes())]
mean_neighbors = mean(num_neighbors)
median_neighbors = np.median(num_neighbors)

In [118]:
pd.DataFrame(numbers_group)[pd.DataFrame(numbers_group)['title'] == 'The Darkest Minds (2018)']

Empty DataFrame
Columns: [title, num_neighbors]
Index: []

In [119]:
print("Mean # of Neighbors per User: ", mean_neighbors)
print("Median # of Neighbors per User: ", median_neighbors)

Mean # of Neighbors per User:  468.20760114425826
Median # of Neighbors per User:  262.5


In our network, users had an average of 94.75 neighbors - or users that they were connected to who watched and rated 15 or more of the same movies. This number is much higher than the median of 60. This suggests that there are number of users who have watched so many movies that they had a larger number of users who watched and rated at least 20 of the same movies.

<b>e. What is the largest number of neighbors that any particular node has?

In [120]:
max_neighbors = max(num_neighbors)
max_neighbors

3426

The largest number of neighbors that any particular node has is 595 which represents

<b>f. What node(s) has the largest number of neighbors?


In [121]:
nodes_with_max = [user for user in list(G.nodes()) if len(list(net.neighbors(G,user))) == max_neighbors]
nodes_with_max

['Matrix, The (1999)']

User 414 is the only user with the maximum number of neighbors in the network

<b>g. What is the fewest number of neighbors that any particular node has?

In [122]:
min_neighbors = min(num_neighbors)
min_neighbors

19

<b>h. What node(s) has the fewest number of neighbors?

In [123]:
nodes_with_min = [user for user in list(G.nodes()) if len(list(net.neighbors(G,user))) == min_neighbors]
nodes_with_min

['Spies Like Us (1985)', 'District 13 (Banlieue 13) (2004)']

In [124]:
len(nodes_with_min)

2

There were 14 users in our network who only had 1 other user that watched at least the same 20 movies as them.

In [125]:
neighbors_df = pd.DataFrame(numbers_group)

In [126]:
neighbors_df = pd.merge(neighbors_df, degree_centrality_df, how='left').rename(columns={'centrality':'degree'})


In [127]:
neighbors_df = pd.merge(neighbors_df, eigenvector_centrality_df, how='left').rename(columns={'centrality':'eigenvector'})

Here we will filter out the movies that are not included in the network

In [134]:
movies_filtered = movies_expanded_df[movies_expanded_df['title'].isin(m1_sub_titles)]

In [135]:
neighbors_expanded = pd.merge(movies_filtered.loc[:,['genre','title']],neighbors_df, how='left')

In [136]:
neighbors_expanded.sort_values(by='num_neighbors', ascending=False).tail(10)

genre                             title  num_neighbors    degree  \
11215     Thriller                  Insurgent (2015)             20  0.004087   
11214       Sci-Fi                  Insurgent (2015)             20  0.004087   
5055         Drama                     Circus (2000)             20  0.004087   
11213       Action                  Insurgent (2015)             20  0.004087   
5056      Thriller                     Circus (2000)             20  0.004087   
10499  Documentary                    Samsara (2011)             20  0.004087   
7780        Comedy              Spies Like Us (1985)             19  0.003883   
8305        Action  District 13 (Banlieue 13) (2004)             19  0.003883   
8306         Crime  District 13 (Banlieue 13) (2004)             19  0.003883   
8307        Sci-Fi  District 13 (Banlieue 13) (2004)             19  0.003883   

       eigenvector  
11215     0.000569  
11214     0.000569  
5055      0.000415  
11213     0.000569  
5056      0.000415  
10499     0.000902  
7780      0.000378  
8305      0.000378  
8306      0.000378  
8307      0.000378

In [139]:
neighbors_expanded[neighbors_expanded['genre'] == 'Drama'].sort_values(by='degree', ascending=False).head(10)

genre                             title  num_neighbors    degree  \
599   Drama               Forrest Gump (1994)           3419  0.698753   
500   Drama               Pulp Fiction (1994)           3353  0.685265   
535   Drama  Shawshank Redemption, The (1994)           3147  0.643164   
3955  Drama            American Beauty (1999)           3010  0.615165   
1255  Drama             Godfather, The (1972)           2946  0.602085   
2856  Drama        Saving Private Ryan (1998)           2939  0.600654   
208   Drama                 Braveheart (1995)           2831  0.578582   
3850  Drama           Sixth Sense, The (1999)           2815  0.575312   
862   Drama           Schindler's List (1993)           2787  0.569589   
4088  Drama                 Fight Club (1999)           2734  0.558757   

      eigenvector  
599      0.050478  
500      0.049426  
535      0.048215  
3955     0.046755  
1255     0.046215  
2856     0.047290  
208      0.046197  
3850     0.045859  
862      0.045543  
4088     0.044346

In [141]:
neighbors_expanded.sort_values(by='degree', ascending=False)

genre                             title  num_neighbors    degree  \
3619      Thriller                Matrix, The (1999)           3426  0.700184   
3618        Sci-Fi                Matrix, The (1999)           3426  0.700184   
3617        Action                Matrix, The (1999)           3426  0.700184   
601            War               Forrest Gump (1994)           3419  0.698753   
598         Comedy               Forrest Gump (1994)           3419  0.698753   
...            ...                               ...            ...       ...   
10499  Documentary                    Samsara (2011)             20  0.004087   
7780        Comedy              Spies Like Us (1985)             19  0.003883   
8305        Action  District 13 (Banlieue 13) (2004)             19  0.003883   
8306         Crime  District 13 (Banlieue 13) (2004)             19  0.003883   
8307        Sci-Fi  District 13 (Banlieue 13) (2004)             19  0.003883   

       eigenvector  
3619      0.050229  
3618      0.050229  
3617      0.050229  
601       0.050478  
598       0.050478  
...            ...  
10499     0.000902  
7780      0.000378  
8305      0.000378  
8306      0.000378  
8307      0.000378  

[11816 rows x 5 columns]

### Centrality

Some of the questions that are of interest would be:

- What is the most popular movie?
- What is the most popular movie across each genre?
- 

### 6. Conclusion

This analysis was helpful in driving some conclusions regarding a specific construction of the network of users who rated movies. One thing that I gained from this process is that network graphs are a way for us to evaluate different relationships amongst different things, but at the same time - as with other types of analysis problems - the real insights are gleaned from the particular questions that we ask of the data and the specific analyses that we apply to the data. Networks are particularly helpful for evaluating relationships and provide a data structure that is best suited for this type of analysis.